In [ ]:
# import packages
import pandas as pd
import numpy as np
import math
import time 
import ast


# import data file
simu_data = pd.read_csv('simu_data.csv', header = 0)


# specify model parameters
period = 1
price_lst = [99, 199, 299, 399, 499, 599, 699, 799, 899, 999]


# specify initial state: A/B/C
state = {'nA':15,'nB':15,'nC':15}


# initialize the tree: the root node is indexed by 0, the next node is indexed by len(tree)
fashion_Tree = {0:{'Week':1,'Child':[],'State':state,'Type':'D','n':0,'V':0}}


# define a function to expand a tree from a given decision node
# add ten state-of-nature nodes (pricing options) to the tree when the focal node has no children
def Expand(tree, node):
    if len(tree[node]['Child']) == 0:
        for price in price_lst:
            tree[len(tree)] = {'Week':tree[node]['Week'],'Child':[],'Parent':node,'Price':price,'Type':'S','n':0,'V':0,'UCB':float('inf')}
        for child_node in range(10):
            tree[node]['Child'].append(len(tree)-(10-child_node))

            
# define a function to perform MCTS algorithm k times from a given decision node of the tree
def BuildTree(tree, node, k):
    season = 0 
    for s in range(k):
        season += 1
        if season > 50:
            season = 1

        # point to the given decision node
        pointer = node

        # extract the state of the given decision node
        week = tree[node]['Week'] 
        nA = tree[node]['State']['nA']
        nB = tree[node]['State']['nB']
        nC = tree[node]['State']['nC']

        # prepare a notebook to keep track of the products sold at given any initial state
        notebook = {}
        while week <= 10:
            notebook[week] = {'Revenue':0,'Delta_nA':0,'Delta_nB':0,'Delta_nC':0}
            week += 1

        
        # start expansion, selection and simulation, continue until reaching the end of a season or the products have been sold out
        week = tree[node]['Week']
        while week <= 10:
            # expansion: expand if the given decision node has no child nodes
            if len(tree[pointer]['Child']) == 0:
                Expand(tree, pointer)

            # selection: select and point to the child node with the highest UCB value, if there are multiple child nodes haven't been visited, select the first one with infinite UCB value
            Max_UCB = 0
            for n in range(10):
                if tree[tree[pointer]['Child'][n]]['UCB'] > Max_UCB:
                    Max_UCB = tree[tree[pointer]['Child'][n]]['UCB']
                    index = n
            pointer = tree[pointer]['Child'][index]


            # simulation
            # for product A
            SKU = 'A'
            A_value = simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) & (simu_data['SKU'] == SKU)]['WTP'] 
            A_value = np.array(A_value)
            A_SC_B_value=simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) &(simu_data['SKU'] == SKU)& (simu_data['WTP'] >= tree[pointer]['Price'] )]['SC_B']
            A_SC_B_value = np.array(A_SC_B_value)
            A_SC_C_value=simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) &(simu_data['SKU'] == SKU)&  (simu_data['WTP'] >= tree[pointer]['Price'] )]['SC_C']
            A_SC_C_value = np.array(A_SC_C_value)
    
            # check how many product A can be sold in this week
            if tree[tree[pointer]['Parent']]['State']['nA'] <= np.sum(tree[pointer]['Price'] <= A_value):
                delta_nA = tree[tree[pointer]['Parent']]['State']['nA']     
            else:
                delta_nA = np.sum(tree[pointer]['Price'] <= A_value)
                
           
            # for product B
            SKU = 'B'
            B_value = simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) & (simu_data['SKU'] == SKU)]['WTP']
            B_value = np.array(B_value)
            B_SC_A_value=simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) &(simu_data['SKU'] == SKU)& (simu_data['WTP'] >= tree[pointer]['Price'] )]['SC_A']
            B_SC_A_value = np.array(B_SC_A_value)
            B_SC_C_value=simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) & (simu_data['SKU'] == SKU)&(simu_data['WTP'] >= tree[pointer]['Price'] )]['SC_C']
            B_SC_C_value = np.array(B_SC_C_value)

            # check how many product B can be sold in this week
            if tree[tree[pointer]['Parent']]['State']['nB'] <= np.sum(tree[pointer]['Price'] <= B_value):
                delta_nB = tree[tree[pointer]['Parent']]['State']['nB']
            else:
                delta_nB = np.sum(tree[pointer]['Price'] <= B_value)
    

            # for product C
            SKU = 'C'
            C_value = simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) & (simu_data['SKU'] == SKU)]['WTP']
            C_value = np.array(C_value)
            C_SC_A_value=simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) &(simu_data['SKU'] == SKU)& (simu_data['WTP'] >= tree[pointer]['Price'] )]['SC_A']
            C_SC_A_value = np.array(C_SC_A_value)
            C_SC_B_value=simu_data[(simu_data['Season'] == season) & (simu_data['Week'] == week) & (simu_data['SKU'] == SKU)&(simu_data['WTP'] >= tree[pointer]['Price'] )]['SC_B']
            C_SC_B_value = np.array(C_SC_B_value)

            # check how many product C can be sold in this week
            if tree[tree[pointer]['Parent']]['State']['nC'] <= np.sum(tree[pointer]['Price'] <= C_value):
                delta_nC = tree[tree[pointer]['Parent']]['State']['nC']
            else:
                delta_nC = np.sum(tree[pointer]['Price'] <= C_value)
                

            # now get the updated system state after the first choice:
            nA = tree[tree[pointer]['Parent']]['State']['nA'] - delta_nA
            nB = tree[tree[pointer]['Parent']]['State']['nB'] - delta_nB
            nC = tree[tree[pointer]['Parent']]['State']['nC'] - delta_nC
                
                
            ##if the first choice is sold out,customers may choose second choice
            #NA_buy\NB_buy\NC_buy means the number of people whose first choice is sold out
            NA_buy=np.sum(tree[pointer]['Price'] <= A_value)-delta_nA
            NB_buy=np.sum(tree[pointer]['Price'] <= B_value)-delta_nB
            NC_buy=np.sum(tree[pointer]['Price'] <= C_value)-delta_nC
               
            
            if NA_buy > 0 and nB > 0:
                second_sold = min(nB, np.sum(A_SC_B_value), NA_buy)
                delta_nB += second_sold
                NA_buy = max(0, NA_buy - second_sold)
                         
            if NA_buy>0 and nC>0:
                second_sold = min(nC, np.sum(A_SC_C_value), NA_buy)
                delta_nC += second_sold
                NA_buy = max(0, NA_buy - second_sold)
              
            if NB_buy>0 and nA>0:
                second_sold = min(nA, np.sum(B_SC_A_value), NB_buy)
                delta_nA += second_sold
                NB_buy = max(0, NB_buy - second_sold)
                
            if NB_buy>0 and nC>0:
                second_sold = min(nC, np.sum(B_SC_C_value), NB_buy)
                delta_nC += second_sold
                NB_buy = max(0, NB_buy - second_sold)
                    
            if NC_buy>0 and nB>0:      
                second_sold = min(nB, np.sum(C_SC_B_value), NC_buy)
                delta_nB += second_sold
                NC_buy = max(0, NC_buy - second_sold)
                
        
            # update notebook
            revenue = tree[pointer]['Price'] * (delta_nA + delta_nB + delta_nC)
            notebook[week]['Revenue'] = revenue
            notebook[week]['Delta_nA'] = delta_nA
            notebook[week]['Delta_nB'] = delta_nB
            notebook[week]['Delta_nC'] = delta_nC

            # now get the updated system state at the end of the week
            nA = tree[tree[pointer]['Parent']]['State']['nA'] - notebook[week]['Delta_nA']
            nB = tree[tree[pointer]['Parent']]['State']['nB'] - notebook[week]['Delta_nB']
            nC = tree[tree[pointer]['Parent']]['State']['nC'] - notebook[week]['Delta_nC']

            # check if the products are sold out
            if (nA + nB + nC) == 0:
                break

            # check if the state has been realized before
            n_Child = len(tree[pointer]['Child'])
            while n_Child > 0:
                if tree[tree[pointer]['Child'][n_Child - 1]]['State'] == {'nA':nA,'nB':nB,'nC':nC}:
                    pointer = tree[pointer]['Child'][n_Child - 1]
                    break
                else:
                    n_Child = n_Child - 1
                
            # if no, add a new decision node
            if n_Child == 0:
                tree[len(tree)] = {'Week':week+1,'Child':[],'Parent':pointer,'State':{'nA':nA,'nB':nB,'nC':nC},'Type':'D','n':0,'V':0} 
                tree[pointer]['Child'].append(len(tree)-1)
                pointer = len(tree) - 1

            # update the current system time
            week = tree[pointer]['Week']

        # start backpropagation (BSR is the total revenue going forward in the simulation)
        BSR = 0
        while pointer >= node:

            # only update BSR if the node is a state-of-nature node (representing a pricing option)
            if tree[pointer]['Type'] == 'S':
                BSR = BSR + notebook[tree[pointer]['Week']]['Revenue']

            # update V and n
            tree[pointer]['V'] = (tree[pointer]['V'] * tree[pointer]['n'] + BSR) / (tree[pointer]['n'] + 1)
            tree[pointer]['n'] = tree[pointer]['n'] + 1

            # update UCB for the state-of-nature node
            # update Ni and UCB for other state-of-nature nodes not chosen in the simulation
            
            if tree[pointer]['Type'] == 'S':
                tree[pointer]['UCB'] = tree[pointer]['V'] + 2 * (math.log(tree[tree[pointer]['Parent']]['n'] + 1) / tree[pointer]['n']) ** 0.5
                for other in tree[tree[pointer]['Parent']]['Child']:
                    if tree[other] != tree[pointer] and tree[other]['n'] > 0:
                        tree[other]['UCB'] = tree[other]['V'] + 2 * (math.log(tree[tree[other]['Parent']]['n'] + 1) / tree[other]['n']) ** 0.5

            if pointer != node:
                pointer = tree[pointer]['Parent']
            else:
                break
            

# define a function for optimization with the built tree
def Optimization(tree, k):
    node = 0
    while int(tree[node]['Week']) <= 10:
        print('Now is week ' + str(tree[node]['Week']) + '.')

        if tree[node]['n'] == 0:
            BuildTree(tree, node, k)

        Max_V = 0
        for n in range(10):
            if tree[tree[node]['Child'][n]]['V'] > Max_V:
                Max_V = tree[tree[node]['Child'][n]]['V']
                index = n
        node = tree[node]['Child'][index]

        print('For the next week, the price will be ' + str(tree[node]['Price']) + '.')
        print('During the upcoming week: ')
        delta_nA = int(input('How many product A are sold? '))
        delta_nB = int(input('How many product B are sold? '))
        delta_nC = int(input('How many product C are sold? '))

        if tree[tree[node]['Parent']]['State']['nA'] <= delta_nA:
            delta_nA = tree[tree[node]['Parent']]['State']['nA']

        if tree[tree[node]['Parent']]['State']['nB'] <= delta_nB:
            delta_nB = tree[tree[node]['Parent']]['State']['nB']

        if tree[tree[node]['Parent']]['State']['nC'] <= delta_nC:
            delta_nC = tree[tree[node]['Parent']]['State']['nC']

        nA = tree[tree[node]['Parent']]['State']['nA'] - delta_nA
        nB = tree[tree[node]['Parent']]['State']['nB'] - delta_nB
        nC = tree[tree[node]['Parent']]['State']['nC'] - delta_nC
        
        Revenue=(delta_nA+delta_nB+delta_nC)*tree[node]['Price']
        
        print('Revenue of this week is:'+str(Revenue))

        if nA + nB + nC == 0:
            print('Sold Out! Yeah!')
            break
        else:
            n_Child = len(tree[node]['Child'])
            while n_Child > 0:
                if tree[tree[node]['Child'][n_Child-1]]['State'] == {'nA':nA,'nB':nB, 'nC': nC}:
                    node = tree[node]['Child'][n_Child-1]
                    break
                else:
                    n_Child = n_Child - 1
            if n_Child == 0:
                tree[len(tree)] = {'Week':int(tree[node]['Week']) + 1,'Child':[],'Parent':node,'State':{'nA':nA,'nB':nB,'nC':nC},'Type':'D','n':0,'V':0} 
                tree[node]['Child'].append(len(tree) - 1)
                node = len(tree) - 1
                
# train the tree
n = 100000
time1 = time.time()
BuildTree(fashion_Tree,0,n)
time2 = time.time()
print(time2 - time1)

# write down the tree in a CSV file
tree_data = pd.DataFrame(data = fashion_Tree)
transposed_tree = tree_data.T
transposed_tree.to_csv("training_data9.csv")

# load the tree data from a csv file to a dictionary
OldTreeData = pd.read_csv("training_data9.csv", header = 0, index_col = 0)
OldTree = OldTreeData.T.to_dict()

for i in range(len(OldTree)):
    OldTree[i]['Child'] = ast.literal_eval(OldTree[i]['Child'])
    if OldTree[i]['Type'] == 'D':
        OldTree[i]['State'] = ast.literal_eval(OldTree[i]['State'])
        OldTree[i].pop('Price')
        OldTree[i].pop('UCB')
    else:
        OldTree[i].pop('State')
        
        
        
# start optimization
m = 5000
time1 = time.time()
Optimization(OldTree,5000)
time2 = time.time()
print(time2 - time1)


852.480161190033
Now is week 1.
For the next week, the price will be 99.0.
During the upcoming week: 
